In [1]:
import jax
jax.config.update('jax_platforms', 'cpu')
import bmi
from minde.minde import MINDE
from minde.scripts.helper import get_data_loader, get_default_config

/home/alberto/miniconda3/envs/minde/lib/python3.12/site-packages/jaxtyping/__init__.py:244: UserWarning: jaxtyping version >=0.2.23 should be used with Equinox version >=0.11.1
  warnings.warn(
/home/alberto/miniconda3/envs/minde/lib/python3.12/site-packages/lightning_fabric/__init__.py:41: Module minde was already imported from None, but /home/alberto/minde is being added to sys.path


In [2]:
bmi.benchmark.BENCHMARK_TASKS

{'1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f14ef336720>,
 'normal_cdf-1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f14ee53e240>,
 '1v1-additive-0.1': <bmi.benchmark.task.Task at 0x7f14eeafa4e0>,
 '1v1-additive-0.75': <bmi.benchmark.task.Task at 0x7f14ee18ebd0>,
 '1v1-bimodal-0.75': <bmi.benchmark.task.Task at 0x7f14ee18f8f0>,
 'wiggly-1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f14ee18ec60>,
 'half_cube-1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f14ee18fec0>,
 'student-identity-1-1-1': <bmi.benchmark.task.Task at 0x7f14ee274140>,
 'asinh-student-identity-1-1-1': <bmi.benchmark.task.Task at 0x7f14ee3189b0>,
 'swissroll_x-normal_cdf-1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f14ee1b4560>,
 'multinormal-dense-2-2-0.5': <bmi.benchmark.task.Task at 0x7f14f2cec2c0>,
 'multinormal-dense-3-3-0.5': <bmi.benchmark.task.Task at 0x7f14ee1b4ce0>,
 'multinormal-dense-5-5-0.5': <bmi.benchmark.task.Task at 0x7f14ee1b4d40>,
 'multinormal-dense-25-25-0.5': <bmi.benchmark.tas

In [3]:
bmi.benchmark.BENCHMARK_TASKS
#name_task = "1v1-normal-0.75"
name_task="1v1-normal-0.75"
task = bmi.benchmark.BENCHMARK_TASKS[name_task]
task.mutual_information

0.41333928659223396

In [4]:
args = get_default_config()
args.type ="c"
args.test_epoch=2
args.max_epochs=25
args.warmup_epochs = 0
args.bs =512
args.lr = 1e-3
args.arch = "mlp"
args.importance_sampling = True
args.use_ema = True
args.input_dim = 2


In [5]:
X, Y = task.sample(1000, seed=42)
var_list = {}

In [6]:
X.shape

(1000, 1)

In [7]:
var_list = {"x": 1, "y": 1}
model = MINDE(args, var_list=var_list)
model(X, Y)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx